In [1]:
import pandas as pd
import numpy as np
from numpy import load
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from scipy.linalg import norm
from PIL import Image, ImageOps
from numpy import asarray
import os

In [2]:
### import data from csv (written in data_processing)
q_array = load('q_array0.npy')
a_array = load('a_array0.npy')
N = 100

### find number of trials
data_trials = int(a_array.shape[0])

In [3]:
xy_array = np.zeros((N,288,432))
xz_array = np.zeros((N,288,432))
# img4 = Image.open('/home/bowen/Documents/Rod_manipulation/Flexible-Tool/png_data/'+str(3)+'_xy.png')
# img4.show()

for i in range(N):
    
    img1 = Image.open('/home/bowen/Documents/Rod_manipulation/Flexible-Tool/png_data/'+str(i)+'_xy.png')
    img2 = Image.open('/home/bowen/Documents/Rod_manipulation/Flexible-Tool/png_data/'+str(i)+'_xz.png')
    img1 = ImageOps.grayscale(img1)
    img2 = ImageOps.grayscale(img2)
    
    xy_array[i,:,:] = asarray(img1)
    xz_array[i,:,:] = asarray(img2)

In [4]:
print(xy_array.shape)
print(xz_array.shape)

(100, 288, 432)
(100, 288, 432)


In [5]:
## put two image arraies into one single array with two channels and then reshape it because of the format of pytorch
image_array = np.stack([xy_array,xz_array], axis = -1).squeeze()
image_array = np.transpose(image_array,[0,3,1,2]) ## change the axis
print(image_array.shape)

(100, 2, 288, 432)


In [6]:
## split image data and a_array into test and train set and put them into tensor and dataloader
image_train_x = (image_array[0:int(0.8*N),:,:,:]).astype(np.float32) 
image_test_x = (image_array[int(0.8*N):N,:,:,:]).astype(np.float32) 

train_y = (a_array[0:int(0.8*N),:]).astype(np.float32)
test_y = (a_array[int(0.8*N):N,:]).astype(np.float32)

y_test_tensor = torch.from_numpy(test_y)
y_train_tensor = torch.from_numpy(train_y)

imagex_test_tensor = torch.from_numpy(image_test_x)
test_data = [(imagex_test_tensor[i],y_test_tensor[i]) for i in range(imagex_test_tensor.shape[0])]
imagex_train_tensor = torch.from_numpy(image_train_x)




In [7]:
train_dataset = torch.utils.data.TensorDataset(imagex_train_tensor, y_train_tensor)
test_dataset = torch.utils.data.TensorDataset(imagex_test_tensor, y_test_tensor)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size = 5, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size = 5, shuffle=True)

for batch, (img,y) in enumerate(train_loader):
    print("batch", batch)
    print("Shape of img: ", img.shape, img.dtype)
    print("Shape of y: ", y.shape, y.dtype)
    break

batch 0
Shape of img:  torch.Size([5, 2, 288, 432]) torch.float32
Shape of y:  torch.Size([5, 6]) torch.float32


In [8]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.pool = nn.MaxPool2d(2)
        self.relu = nn.functional.relu

        self.conv1 = nn.Conv2d(in_channels=2, out_channels=16, kernel_size=3)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3)
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3)
        self.conv4 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3)
        self.conv5 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3)
        
        ## the input features should be the flatten version of the outchannel from the last conv2d layer
        self.linear1 = nn.Linear(9856, 4096)
        self.linear2 = nn.Linear(4096, 1024)
        self.linear3 = nn.Linear(1024, 256)
        self.linear4 = nn.Linear(256, 64)
        self.linear5 = nn.Linear(64, 32)
        self.linear6 = nn.Linear(32, 6)

    def forward(self, img):
        img_out = self.pool(self.relu(self.conv1(img)))
        img_out = self.pool(self.relu(self.conv2(img_out)))
        img_out = self.pool(self.relu(self.conv3(img_out)))
        img_out = self.pool(self.relu(self.conv4(img_out)))
        img_out = self.pool(self.relu(self.conv5(img_out)))

        img_out = nn.Flatten()(img_out)
#         print(img_out.size())

        output = self.relu(self.linear1(img_out))
        output = self.relu(self.linear2(output))
        output = self.relu(self.linear3(output))
        output = self.relu(self.linear4(output))
        output = self.relu(self.linear5(output))
        output = self.linear6(output)

        return output
model = Net()
print(model)

Net(
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv1): Conv2d(2, 16, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv4): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (conv5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
  (linear1): Linear(in_features=9856, out_features=4096, bias=True)
  (linear2): Linear(in_features=4096, out_features=1024, bias=True)
  (linear3): Linear(in_features=1024, out_features=256, bias=True)
  (linear4): Linear(in_features=256, out_features=64, bias=True)
  (linear5): Linear(in_features=64, out_features=32, bias=True)
  (linear6): Linear(in_features=32, out_features=6, bias=True)
)


In [9]:
### loss function and optimizer for training 
loss_fn = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)
       

In [10]:
### Functions for training
def train(dataloader, model, loss_fn, optimizer,loss_list):
    size = len(dataloader.dataset)
    model.train()
    for batch, (img,y) in enumerate(dataloader):

#         print("batch ", batch)
#         print("Shape of img: ", img.size())
#         print("Shape of q: ", q.size())
    
        # Compute prediction error
        pred = model(img)
#         print("pred", pred)
#         print("y", y)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 10 == 0:
            loss, current = loss.item(), batch * 9856
            loss_list.append(loss)
        if batch % 20 == 0:
#             print('-----------')
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
    return loss_list

In [11]:
### function for testing the model
def test(dataloader, model,test_loss_list):
    size = len(dataloader.dataset)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for img, y in dataloader:
            pred = model(img)
            test_loss += loss_fn(pred, y)
    print(f"Test Error: Loss = {test_loss:>8f} \n")
    test_loss_list.append(test_loss)
    return test_loss_list

In [12]:
### Train the model
epochs = 100
loss_list=[]
test_loss_list=[]
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    loss_list = train(train_loader, model, loss_fn, optimizer,loss_list)
#     print('finish train')
    test_loss_list = test(test_loader, model, test_loss_list)

Epoch 1
-------------------------------
loss: 0.523851  [    0/   80]
Test Error: Loss = 20.983765 

Epoch 2
-------------------------------
loss: 5.477130  [    0/   80]
Test Error: Loss = 2.066396 

Epoch 3
-------------------------------
loss: 0.501723  [    0/   80]
Test Error: Loss = 0.750751 

Epoch 4
-------------------------------
loss: 0.123328  [    0/   80]
Test Error: Loss = 0.620464 

Epoch 5
-------------------------------
loss: 0.202047  [    0/   80]
Test Error: Loss = 0.533228 

Epoch 6
-------------------------------
loss: 0.142782  [    0/   80]
Test Error: Loss = 0.539125 

Epoch 7
-------------------------------
loss: 0.127401  [    0/   80]
Test Error: Loss = 0.509082 

Epoch 8
-------------------------------
loss: 0.111336  [    0/   80]
Test Error: Loss = 0.502470 

Epoch 9
-------------------------------
loss: 0.095121  [    0/   80]
Test Error: Loss = 0.492270 

Epoch 10
-------------------------------
loss: 0.126820  [    0/   80]
Test Error: Loss = 0.456433 

loss: 0.108913  [    0/   80]
Test Error: Loss = 0.394989 

Epoch 83
-------------------------------
loss: 0.098208  [    0/   80]
Test Error: Loss = 0.396512 

Epoch 84
-------------------------------
loss: 0.102238  [    0/   80]
Test Error: Loss = 0.406289 

Epoch 85
-------------------------------
loss: 0.115405  [    0/   80]
Test Error: Loss = 0.404804 

Epoch 86
-------------------------------
loss: 0.090346  [    0/   80]
Test Error: Loss = 0.390193 

Epoch 87
-------------------------------
loss: 0.078137  [    0/   80]
Test Error: Loss = 0.391907 

Epoch 88
-------------------------------
loss: 0.069825  [    0/   80]
Test Error: Loss = 0.392982 

Epoch 89
-------------------------------
loss: 0.080013  [    0/   80]
Test Error: Loss = 0.402678 

Epoch 90
-------------------------------
loss: 0.070545  [    0/   80]
Test Error: Loss = 0.392843 

Epoch 91
-------------------------------
loss: 0.074435  [    0/   80]
Test Error: Loss = 0.391312 

Epoch 92
----------------------

In [13]:
n = np.random.randint(0,int(0.2*N))
img,y = test_dataset[n]
img = torch.unsqueeze(img, 0) 
# q = torch.unsqueeze(q, 0) 
pred = model(img)
print(pred[0])
print(y_test_tensor[n])

tensor([0.4783, 0.5129, 0.5357, 0.5211, 0.4849, 0.5253],
       grad_fn=<SelectBackward>)
tensor([0.7901, 0.5800, 0.4564, 0.7029, 0.4793, 0.7343])
